# Oveview

Een view in de `views.py` krijgt een request via de browser. En zal dan via een specifieke "template" iest tonen op het scherm.
Een view is de manier om content op een webpagina te tonen.


In onze "poll"-app zullen we de volgende views hebben:

* Questions overview page   -> toont een overzicht van de vragen, de laatste eerst (bestaat al met "index")
* Question detail page      -> een detail van de vraag
* Question result page      -> toont de antwoorden voor een vraag
* Vote action               -> de user kan stemmen

Ook belangrijk is om te weten dat de "urlconfs" in de `urls.py` verwijzen naar een view. 
Dus wanneer een gebruiker naar een url gaat, zal django kijken in de `urls.py` of deze hierin voorkomt. Deze zal dan naar een view verwijzen. En de logica binnen deze view zal dan duidelijk maken wat we zullen tonen op de webpagina voor deze url.

## views

Laten we 3 extra views schrijven.

In [ ]:
def detail(request, question_id):
    return HttpResponse(f"You're looking at question {question_id}.")

def results(request, question_id):
    response = f"You're looking at the results of question {question_id}."
    return HttpResponse(response)

def vote(request, question_id):
    return HttpResponse(f"You're voting on question {question_id}.")

Nu moeten we ook nog de `urls.py`-file aanpassen zodat de juiste urls worden gelinkt aan deze views.

In [ ]:
from django.urls import path
from . import views

urlpatterns = [
    path("", views.index, name="index"),
    path("<int:question_id>/", views.detail, name="detail"),
    path("<int:question_id>/results/", views.results, name="results"),
    path("<int:question_id>/vote/", views.vote, name="vote"),
]

Als we nu naar `http://127.0.0.1:8000/polls/1/` gaan zullen we de pagina zien.

We kunnen ook naar `http://127.0.0.1:8000/polls/1/results` en `http://127.0.0.1:8000/polls/1/vote` kijken.


Ook urls met een question die nog niet in de databank zit zal werken:
``http://127.0.0.1:8000/polls/34/``

## Kleine zijsprong

Vooraleer we verder gaan met de bestaande views up te daten, even een view maken om wat zaken te testen met de database.

in views:

In [ ]:
def test_db(request):
    print("--- Dit is een test ---")
    return HttpResponse("Check the print on your server")

voeg toe in urls:

In [ ]:
path("test_db", views.test_db, name="test_db"),


Nu kan je naar `http://127.0.0.1:8000/polls/test_db` gaan.

Kijk nu in de terminal van je server en dan zal je onze print statement zien.

We passen nu de view aan zodat we de resultaten van een aantal database actie kunnen zien.

In [ ]:
from polls.models import Choice, Question 

def test_db(request):
    print("--- Dit is een test ---")

    all_question = Question.objects.all()
    print(all_question)
    
    return HttpResponse("Check the print on your server")

Refresh uw pagina, en dan zal je in de terminal de Queryset zien.

Nu zullen we element saven in de database.

In [ ]:
def test_db(request):
    print("--- Dit is een test ---")

    all_question = Question.objects.all()
    print(all_question)

    q = Question(question_text="What's new?", pub_date=timezone.now())
    q.save()

    return HttpResponse("Check the print on your server")

Als we nu onze pagina (url) refreshen, dan zullen we in onze database (db browser) een nieuwe Question zien. 

Elke keer we de pagina refreshen zal er een nieuwe toevoeging zijn in de db.

We kunnen eenvoudig de data uit de kolommen capteren:

In [ ]:
def test_db(request):
    print("--- Dit is een test ---")

    all_question = Question.objects.all()
    print(all_question)

    q = Question(question_text="What's new?", pub_date=timezone.now())
    q.save()

    print(f"Question aangemaakt met id: {q.id}, text: {q.question_text}, datum: {q.pub_date}")

    return HttpResponse("Check the print on your server")


We kunnen de data ook eenvoudig aanpassen.

We gaan nu een extra methode toevoegen aan onze Model-klasse "Question".

In [ ]:
from django.utils import timezone
import datetime

        
def was_published_recently(self):
    return self.pub_date >= timezone.now() - datetime.timedelta(days=1)

We kunnen deze methode aanspreken in onze view.

In [ ]:
def test_db(request):
    print("--- Dit is een test ---")

    all_question = Question.objects.all()
    print(all_question)

    q = Question(question_text="What's new?", pub_date=timezone.now())
    q.save()

    print(f"Question aangemaakt met id: {q.id}, text: {q.question_text}, datum: {q.pub_date}")

    q.question_text = "We hebben de vraag veranderd ... wie weet het nu nog ?"
    q.save()

    print(f"de text van de vraag met id {q.id} is veranderd naar '{q.question_text}'")

    all_question = Question.objects.all()
    print(all_question)

    print("--------- Recent Published ---------")
    print(q.was_published_recently())

    return HttpResponse("Check the print on your server")

We voegen nog een aantal extra code toe om te filteren in onze data.
We stoppen met telkens nieuwe data toe te voegen in de db.

In [ ]:
def test_db(request):
    print("--- Dit is een test ---")

    all_question = Question.objects.all()
    print(all_question)

    # q = Question(question_text="What's new?", pub_date=timezone.now())
    # q.save()

    # print(f"Question aangemaakt met id: {q.id}, text: {q.question_text}, datum: {q.pub_date}")

    # q.question_text = "We hebben de vraag veranderd ... wie weet het nu nog ?"
    # q.save()

    # print(f"de text van de vraag met id {q.id} is veranderd naar '{q.question_text}'")

    # all_question = Question.objects.all()
    # print(all_question)

    # print("--------- Recent Published ---------")
    # print(q.was_published_recently())

    q_1 = Question.objects.filter(id=1)
    print(f"We hebben de Quetion met id: {q_1.id} opgehaald")

    return HttpResponse("Check the print on your server")

We krijgen nu een error.

`Exception Type:	AttributeError`
`Exception Value: 'QuerySet' object has no attribute 'id'`

als we onze print uit commenten. En we printen `q_1` uit zien we het volgende:


In [ ]:

def test_db(request):
    print("--- Dit is een test ---")

    all_question = Question.objects.all()
    print(all_question)

    # ...

    q_1 = Question.objects.filter(id=1)
    # print(f"We hebben de Quetion met id: {q_1.id} opgehaald")
    print(q_1)

    return HttpResponse("Check the print on your server")

`<QuerySet [<Question: What's up?>]>`

filter geeft niet 1 element terug, maar een "QuerySet" met alle elementen die aan de filter voldoen.

om exact 1 specifiek element te krijgen, gebruiken we `get`

In [ ]:

def test_db(request):
    print("--- Dit is een test ---")

    all_question = Question.objects.all()
    print(all_question)

    # ...

    q_1 = Question.objects.filter(id=1)
    # print(f"We hebben de Quetion met id: {q_1.id} opgehaald")
    print(q_1)

    q_2 = Question.objects.get(id=2)
    print(f"We hebben de Question met id: {q_2.id} opgehaald")

    return HttpResponse("Check the print on your server")

Nog een aantal mogelijkheden met filter.


In [ ]:
def test_db(request):
    print("--- Dit is een test ---")

    all_question = Question.objects.all()
    print(all_question)

    # ...

    qs = Question.objects.filter(question_text__startswith="What")
    print(qs)

    return HttpResponse("Check the print on your server")

Nu krijgen we een Queryset met alle Questions die starten met "What"

In [ ]:
def test_db(request):
    print("--- Dit is een test ---")

    all_question = Question.objects.all()
    print(all_question)

    # ...

    current_year = timezone.now().year
    qs_2 = Question.objects.filter(pub_date__year=current_year)
    print(qs_2)


    return HttpResponse("Check the print on your server")

Nu krijgen we alle vragen van die dit jaar zijn toegevoegd

In [ ]:
def test_db(request):
    print("--- Dit is een test ---")

    all_question = Question.objects.all()
    print(all_question)

    # ...


    previous_year = (timezone.now()- datetime.timedelta(weeks=52)).year
    qs_3 = Question.objects.filter(pub_date__year=previous_year)
    print(qs_3)

    return HttpResponse("Check the print on your server")

Nu krijgen we alle vragen die vorig jaar zijn toegevoegd. 

Je kan via de admin de datum van 1 vraag veranderen om te dubbel checken.

Nu gaan we de choices voor een bepaalde question checken.

We gebruiken `_set` om de gerelateerde info op te halen.

In [ ]:
def test_db(request):
    print("--- Dit is een test ---")

    all_question = Question.objects.all()
    print(all_question)

    # ...

    q_3 = Question.objects.get(pk=1)
    choices = q_3.choice_set.all()
    print(choices)

    return HttpResponse("Check the print on your server")

We zien dat de Queryset nog leeg is. 

We hebben ook nog geen choices aangemaakt. Dat doen we nu:

In [ ]:
def test_db(request):
    print("--- Dit is een test ---")

    all_question = Question.objects.all()
    print(all_question)

    # ...

    q_3.choice_set.create(choice_text="Not much", votes=0)
    q_3.choice_set.create(choice_text="The sky", votes=0)
    c = Choice(question=q_3, choice_text="Just hacking again", votes=0)
    c.save()

    print(c.question)


    return HttpResponse("Check the print on your server")

We printen nu de 3 choices uit van onze question.

In [ ]:
def test_db(request):
    print("--- Dit is een test ---")

    all_question = Question.objects.all()
    print(all_question)

    # ...

    choices = q_3.choice_set.all()
    print(choices)

    print(q_3.choice_set.count())


    return HttpResponse("Check the print on your server")

Nu printen we de choices uit van de questions uit het vorige jaar.

In [ ]:
def test_db(request):
    print("--- Dit is een test ---")

    all_question = Question.objects.all()
    print(all_question)

    # ...

    previous_year = (timezone.now()- datetime.timedelta(weeks=52)).year
    choices_from_question_current_year = Choice.objects.filter(question__pub_date__year=previous_year)
    print(choices_from_question_current_year)


    return HttpResponse("Check the print on your server")

tot slotte deleten we één van de choices.

In [ ]:
def test_db(request):
    print("--- Dit is een test ---")

    all_question = Question.objects.all()
    print(all_question)

    # ...
    c_1 = q_3.choice_set.filter(choice_text__startswith="Just ")
    c_1.delete()

    return HttpResponse("Check the print on your server")

## Terug naar onze views.

We gaan op onze index-view de laatste 3 Questions tonen.

In [ ]:
def index(request):
    latest_question_list = Question.objects.order_by("-pub_date")[:3]
    output = ", <br>".join([q.question_text for q in latest_question_list])
    return HttpResponse(output)

Dit is nartuurlijk niet de meest elegante manier. De gegevens zijn hard-coded in een string.

We zullen nu het Template-systeem van Django gebruiken om dit eleganter te maken.


We maken eerst in onze app de folders : `/templates/polls` aan

Dit moet op deze manier worden opgezet, zodat Django de juiste templates kan vinden.

We maken hierin nu de file : `index.html` aan

In [ ]:
<h1>Onze lijst</h1>

Om deze "HTML"-file aan te spreken, veranderen we onze  code in "views.py".

In [ ]:
def index(request):
    # latest_question_list = Question.objects.order_by("-pub_date")[:3]
    # output = ", <br>".join([q.question_text for q in latest_question_list])
    template = loader.get_template("polls/index.html")
    
    return HttpResponse(template.render())

Nu zien we us onze <h1> titel.

Nu zullen we extra context meegeven aan onze HttpResponse. Zodat we deze op een elegante wijze kunnen tonen.

We zullen daarna ook onze HTML aanpassen, om onze context te gebruiken.

In [ ]:
def index(request):
    latest_question_list = Question.objects.order_by("-pub_date")[:3]
    template = loader.get_template("polls/index.html")
    context = {
        "latest_question_list": latest_question_list
    }
    return HttpResponse(template.render(context, request))

In [ ]:
{% if latest_question_list %}
    <ul>
    {% for question in latest_question_list %}
        <li><a href="/polls/{{ question.id }}/">{{ question.question_text }}</a></li>
    {% endfor %}
    </ul>
{% else %}
    <p>No polls are available.</p>
{% endif %}

Je ziet in deze HTML-file een aantal zaken.

* {% %}
* {{ }}
* HTML

**SCHORTCUT**

We kunnen onze view nog iets updaten door `render()` te gebruiken.

In [ ]:
from django.shortcuts import render

def index(request):
    latest_question_list = Question.objects.order_by("-pub_date")[:3]
    context = {
        "latest_question_list": latest_question_list
    }
    return render(request, "polls/index.html", context)

## Detail view 

We maken onze detail view aan in `views.py`:

In [ ]:
def detail(request, question_id):
    question = Question.objects.get(pk=question_id)
    context = {
        "question" : question
    }
    return render(request, "polls/detail.html", context)

en  de `detail.html`:

In [ ]:
{{ question }}

als we nu naar `http://127.0.0.1:8000/polls/1/` gaan zien we de question_text (omdat dat de __str__ representatie is).

wanneer we echter naar `http://127.0.0.1:8000/polls/100/` gaan, dan krijgen we een error te zien.

## -> 404 error

Daarom haan we nu een "error"-handling toevoegen:

In [ ]:
def detail(request, question_id):
    try:
        question = Question.objects.get(pk=question_id)
        context = {
            "question" : question
        }
    except Question.DoesNotExist as e:
        raise Http404(f"Personal note: Question does not exist => \n {e}")
    return render(request, "polls/detail.html", context)

We zien nu een 404-error in de browser.
We zullen deze later nog verder pimpen. (dit heeft ook te maken met de "Debug = True" in de settings.py -file)


Verschillende error codes => https://kb.iu.edu/d/bfrc

===> 404	Not Found	Requested file was not found

**SHORTCUT**

Er bestaat ook een functie die het bovenstaande ineens doet.

In [ ]:
from django.shortcuts import render, get_object_or_404

def detail(request, question_id):
    question = get_object_or_404(Question, pk=question_id)
    context = {
        "question" : question
    }
    return render(request, "polls/detail.html", context)

check opnieuw:

*  `http://127.0.0.1:8000/polls/1/`
*  `http://127.0.0.1:8000/polls/100/` 


er bestaat ook `get_list_or_404()`, deze gebruikt "filter()" zal een 404 geven als de Queryset empty is

## Template system for Detail

we passen onze html aan voor de detail-view.

In [ ]:
<h1>{{question.question_text}}</h1>

<ul>
    {% for choice in question.choice_set.all %}
        <li>{{choice.choice_text}} </li>
    {% endfor %}
</ul>

Als we nu de detail-url open van de question met id==1 zien we het resultaat:

`http://127.0.0.1:8000/polls/1/`

## remove hardcoded urls 

In de `index.html` hebben we de urls naar de detail pagina "hardcoded".

<li><a href="/polls/{{ question.id }}/">{{ question.question_text }}</a></li>

dit betekent dat het nu moeilijk wordt om de urls te veranderen. Want dan moet je op alle plaatsen in de templates de urls gaan veranderen.

Daarom is er de mogelijkheid om deze dynamischer te maken:

In [ ]:
<li><a href="{% url 'detail' question.id %}">{{ question.question_text }}</a></li>

We gebruiken dus {% url %}

waarin 'detail' staat voor de naam die we hebben gegeven in de urls.

In [ ]:
path("<int:question_id>/", views.detail, name="detail"),

Dit is dus veel eenvoudiger om later de urls aan te passen. Je hoeft dit dan enkel in de `urls.py` te doen.

Er is nog 1 ding dat we best aanpassen. 

Wat als er meerdere apps zijn die 'detail' hebben als url_name ?

Daarom voegen we eerst de `app_name = "polls` toe aan de `urls.py`.


In [ ]:
from django.urls import path
from . import views

app_name = "polls"
urlpatterns = [
    # ...
]

En daarna kunnen we in onze template de verwijzing aanpassen:

<li><a href="{% url 'polls:detail' question.id %}">{{ question.question_text }}</a></li>